
# Instability of algorithms

Suppose we wish to compute the integrals 

$$I(n) = \int_0^1 \frac{x^n}{x + 5} \mathrm{d}x , \quad n = 1, 2, \ldots \tag{1}$$

Integrals $I(n)$ have the following properties:

- All $I(n)$ are strictly positive, $I(n) > 0$, since the integrand is positive throughout the interval (0, 1). 

- $I(n)$ are monotonically decreasing as n is increasing: 
since $x^{n+1} < x^n$ for $0 < x < 1$, then $\frac{x^{n+1}}{x+5} < \frac{x^n}{x+5}$, 
and thus $I(n) > I(n+1)$.

You can use those properties of $I(n)$ to check the correctness of your numerical calculations.


Transforming the integrand as follows, 

$$
I(n+1) = \int_0^1 \frac{x^{n+1}}{x + 5} \mathrm{d}x =  
\int_0^1 \frac{x^{n+1} + 5 x^n - 5 x^n}{x + 5} \mathrm{d}x =
\int_0^1 \frac{x^n (x + 5)}{x + 5} \mathrm{d}x  - 5 \int_0^1 \frac{x^n}{x + 5} \mathrm{d}x =
\int_0^1 x^n \mathrm{d}x - 5 \, I(n) = \frac{1}{n+1} - 5 \, I(n) ,
$$

we establish the following recurrence relation:

$$I(n+1) = \frac{1}{n+1} - 5 \,I(n). \tag{2}$$

The value of $I(1)$ which is an elementary integral, is as follows.

$$I(1) = 1 + 5 \, log(5/6) . \tag{3}$$

We can use recurrence relation Eq. (2) together with the initial condition Eq. (3) to calculate $I(n)$ for any $n$.


**The goal of the assignments is to calculate the integrals $I(n)$ 
using just described recurrence algorithm and investigate the algorithm's 
numerical properties.** You check the accuracy 
of the calculations by comparing the algorithm's results with the ones 
obtained by the quadrature `quadgk` from Julia package `QuadGK`.


#### **0**. 

Install (if needed) and load required packages. 

Once new packages are installed, comment out the package installation command(s). (Those commands need to be executed only once but the notebook is expected to be run multiple times.) Do not delete the installation commands.

In [ ]:
# ] add PackageA, PackageB

In [ ]:

using ....
using PyPlot


Define the number of integrals to calculate:

In [ ]:

np = 23


Define a function for the integrand.

In [ ]:

integrand(x, n) = your_code_here

####  **1**. 

Write a function, `quadrature(m)`, that calculates the integrals I(n) 
for $n = 1, 2, ..., m$ using `quadgk` integrator and 
returns a vector of calculated integrals' values.

In [ ]:

"""
     res = quadrature(m)

Evaluate the integrals I(n), Eq. (1) for n = 1:m using quadgk integrator. 
Return a vector of calculated integrals' values.
"""
function quadrature(m)
    r = zeros(m)
    for n = 1:m
        r[n] = quadgk(x -> integrand(x, n), 0.0, 1.0)[1]
    end
    return r
end


Calculate the integrals using `quadrature()`:

In [ ]:

res1 = quadrature(np);


#### **2**.

Write a function, `recurrence(m)`, that calculates the integrals I(n) 
for $n = 1, 2, ..., m$ using the recurrence relations Eq. (2) and 
returns a vector of calculated integrals' values.

In [ ]:

"""
     res = recurrence(m)

Evaluate the integrals I(n), Eq. (1) for n = 1:m using recurrence relations Eq. (2). 
Return a vector of calculated integrals' values.
"""
function recurrence(m)
    r = zeros(m)
    # your code here
    return r
end 


Calculate the integrals using `recurrence()`:

In [ ]:

res2 = recurrence(np);


#### **3**.

Plot the results of calculated by both functions.  Provide the grid, axes labels, title, and the legend for your graph.

In [ ]:

plot(1:np, res1, marker="o", label="QuadGK")
plot(1:np, res2, marker="o", label="Recurrence")
# your code here


Notice that two graphs that are supposed to represent the same function $I(n)$ are rather
different. **Explain** in the cell below which of two functions produced the wrong result:


#### **4**.

To see what is exactly wrong, plot the absolute error of evaluation of $I(n)$. Provide the grid, axes labels, title, and the legend for your graph.

In [ ]:

proper_axes_type_here(1:np, (abs.(res1 .- res2)), marker="o", label="numerical experiment", linestyle="none")
# your code here


You should observe that the error in calculations monotonically grows from about 
$10^{-16}$ for $I(1)$ to about $10^0$ for $I(23)$. 
What caused the large error? Provide your explanation in the cell below:


To confirm your understanding, supplement the previous graph by an
additional plot of the expected absolute error of the algorithm as a function of $n$.

In [ ]:

proper_axes_type_here(1:np, (abs.(res1 .- res2)), marker="o", label="numerical experiment", linestyle="none")
proper_axes_type_here(1:np, eps() * your_code_here, linestyle="dashed", label="Expected absolute error")
# your code here


Does your explanation of the error and your observed error are in agreement? **Explain** in the cell below.


#### **5**.

The enormous magnification of the error is the result of the algorithm we choose to use.

How can we modify the algorithm to avoids this instability?
If we rewrite the recurrence relation Eq. (2) as

$$I(n) = (1/(n+1) - I(n+1))/5, \qquad n = \ldots, 4, 3, 2, 1, \tag{3}$$

then at each stage of the calculation, the error in $I(n)$ is decreased by a factor
of 1/5. So if we start with an arbitrary value for some $I(k)$ and work backbackwards, 
any initial error or rounding errors which occur will be decreased at
each step. This is called a *stable algorithm*.


Write a function, `recurrence_reverse(m)`, that calculates the integrals I(n) 
for $n = 1, 2, ..., m$ using the stable recurrence relations Eq. (3) and 
returns a vector of calculated integrals' values. 

Since the reverse recurrence algorithm starts with an arbitrary value 
for $I(k)$, values of the integral with the largest $I(l)$, $l = k, k-1, k-2, \ldots$ 
contain an arbitrary but fast decreasing error. Your function should start recurrence from $I(l)$, 
$l = m + mextra$, $l > m$,
where $m$ is the required number of integrals, and discard $mextra$ integrals containing errors.
Estimate a suitable `mextra` so that the error in $I(m)$ is less than machine epsilon. Explain your estimate.

In [ ]:

"""
     res = recurrence_reverse(np, extra)

Evaluate the integrals I(n), Eq. (1) for n = 1:(np+extra) using recurrence , 
relation I(n) = (1/(n+1) - I(n+1))/5, with the initial condition I(np+extra)=0
Return a vector of calculated integrals' values discarding the last `extra` elements
"""
function recurrence_reverse(m, mextra)
    r = zeros(m+mextra)
    r[end] = 0.0
    # your code here
    return r[1:m]  # discard the last mextra elements
end 


Calculate the integrals using `recurrence_reverse()`:

In [ ]:

mextra = your_estimate here
res3 = recurrence_reverse(np, mextra);


Plot the absolute error of evaluation of $I(n)$ by the stable algorithm. 
Provide the grid, axes labels, title, and the legend for your graph.

In [ ]:

proper_axes_type_here(1:np, (abs.(res1 .- res3)), marker="o", label="numerical experiment", linestyle="none")
# your code here


Indicate whether errors now stay below machine precision. 